# Evaluation Notebook
A playground notebook for exploration and evaluation of the IEMA operational data. Not directly used in research.

In [ ]:
import numpy as np
import config as cfg
import pickle
import pandas as pd
import os
import networkx as nx
import matplotlib.pyplot as plt

## Load data

In [ ]:
record_timestamp = "20241105_082609"

record_timestamp

In [ ]:
with open("precomputed/features.pkl", "rb") as f:
    df_features = pickle.load(f)
nd_features = df_features.values

nd_features.shape

In [ ]:
with open(
    f"output/{record_timestamp}/{record_timestamp}_analysis_population.pkl",
    "rb",
) as f:
    df_iterations = pickle.load(f)

df_iterations

## Functional Variance (Population Diversity)

In [ ]:
all_pops_cat = []
for pop in df_iterations["pop"].unique():
    df_gen = df_iterations.loc[
        df_iterations["pop"] == pop, ["p1", "p2", "p3", "p4", "p5", "p6"]
    ]

    var = df_gen.var(axis=0)
    all_pops_cat.append(np.mean(var))

np.array(all_pops_cat).shape

In [ ]:
# plot variance
plt.plot(all_pops_cat)
plt.xlabel("Population")
plt.ylabel("Variance")
plt.title("Variance of each population")
plt.ylim(0, 0.2)
plt.savefig(f"output/{record_timestamp}/population_diversity.png")
plt.show()

## Categorical Diversity

In [ ]:
with open("precomputed/filenames.pkl", "rb") as f:
    df_filenames = pickle.load(f)
df_filenames.filename = df_filenames.filename.apply(os.path.splitext).str[0]

df_filenames.head()

In [ ]:
# Load the metadata file

df_metacoll = pd.read_csv(cfg.FSD50K_METADATA_COLLECTION_PATH)
df_metacoll.mids = df_metacoll.mids.str.split(",")
df_metacoll.fname = df_metacoll.fname.astype(str)

df_metacoll.head()

In [ ]:
df_iter_w_filename = df_iterations.merge(
    df_filenames, on="sample_id", how="left"
)
df_iter_w_filename.filename = df_iter_w_filename.filename.astype(str)

df_iter_w_filename.head()

In [ ]:
df_ontology_lookup = df_iter_w_filename.merge(
    df_metacoll, left_on="filename", right_on="fname", how="left"
)
df_ontology_lookup["mids_first"] = df_ontology_lookup.mids.str[0]

df_ontology_lookup.head()

In [ ]:
all_pops_cat = []
for pop in df_ontology_lookup["pop"].unique():
    cats = df_ontology_lookup.loc[
        df_ontology_lookup["pop"] == pop, "mids_first"
    ]
    all_pops_cat.append(len(set(cats)) / len(cats))

np.array(all_pops_cat).shape

In [ ]:
# plot

plt.clf()
plt.plot(all_pops_cat)
plt.xlabel("Population")
plt.ylabel("Mean Category Diversity")
plt.title("Mean category diversity for each population")
plt.ylim(0, 1)
plt.savefig(f"output/{record_timestamp}/category_diversity.png")
plt.show()

## Coverage of Dataset Categories/Samples

In [ ]:
df_ontology_lookup.fname.unique().shape[0], df_metacoll.fname.astype(
    str
).unique().shape[0]

In [ ]:
df_ontology_lookup.fname.unique().shape[0] / df_metacoll.fname.astype(
    str
).unique().shape[0] * 100, "percent"

## Phylogenetic
Graph operations

In [ ]:
# load the graph to nx

with open(
    f"output/{record_timestamp}/{record_timestamp}_analysis_evo_graph.gpickle",
    "rb",
) as f:
    G = pickle.load(f)

G.number_of_nodes()

In [ ]:
# get copt of the graph

G_plot = G.copy()


G_plot.number_of_nodes()

In [ ]:
# make bidirectional by adding the reverse edges

for u, v in G_plot.edges():
    G_plot.add_edge(v, u)

G_plot.number_of_edges()

## Visualize the Graph

In [16]:
# Interactive visualization
%matplotlib qt

In [ ]:
# copy and optionally filter out for testing

G_filtered = G_plot.copy()

# optional, uncomment to filter out below a population threshold
# G_filtered.remove_nodes_from(
#     [node[0] for node in G_plot.nodes(data=True) if node[1]["pop"] < 147] # specify a population threshold
# )

In [ ]:
# Draw the graph multipartite

# get the positions multi-partite
pos = nx.multipartite_layout(G_filtered, subset_key="pop", align="horizontal")

# draw the graph
plt.clf()
nx.draw(
    G_filtered,
    pos,
    node_size=75,
    font_size=10,
    font_weight="bold",
    arrowsize=8,
    width=1,
    alpha=0.75,
)

# add labels
for node in G_filtered.nodes():
    x, y = pos[node]
    label = node
    plt.text(x, y, label, fontsize=12, rotation=45, ha="left")

plt.title("Evolutionary Graph")
plt.show()

## Pairwise Distances

In [ ]:
# get max population
max_pop = df_iterations["pop"].max()

max_pop

In [ ]:
all_mpd_pops = []
all_pops_unique = (
    pd.Series([node[1]["pop"] for node in G_plot.nodes(data=True)])
    .sort_values()
    .unique()
)
for pop in [max_pop]:
    leaf_nodes = [n for n in G_plot.nodes(data=True) if n[1]["pop"] == pop]
    G_upto_pop = G_plot.subgraph(
        [n[0] for n in G_plot.nodes(data=True) if n[1]["pop"] <= pop]
    )

    # for each unique pair of leaf nodes,
    # calculate the shortest path between them
    pairwise_distances = {}
    no_path = []
    for i in range(len(leaf_nodes)):
        for j in range(i + 1, len(leaf_nodes)):
            try:
                dist = nx.shortest_path_length(
                    G_upto_pop, leaf_nodes[i][0], leaf_nodes[j][0]
                )
                if dist > 2:
                    pairwise_distances[(i, j)] = dist + 1
            except nx.NetworkXNoPath:
                no_path.append((i, j))

    pairwise_distances_values = np.array(list(pairwise_distances.values()))

    mpd_pop = (
        pairwise_distances_values.sum()
        * 2
        / (len(leaf_nodes) * (len(leaf_nodes) - 1))
    )
    all_mpd_pops.append(mpd_pop)
    print(
        f"Population {str(pop).zfill(3)} "
        f"Mean Pairwise Distances: {mpd_pop.round(2)} ",
    )

np.array(all_mpd_pops).shape, np.array(no_path).shape

In [ ]:
# plot

plt.clf()
plt.plot(all_mpd_pops)
plt.xlabel("Population")
plt.ylabel("Mean Pairwise Distance")
plt.title("Mean Pairwise Distances for Each Population")
plt.savefig(f"output/{record_timestamp}/mean_pairwise_distance.png")
plt.show()

In [ ]:
final_mpd = all_mpd_pops[-1]

final_mpd

## Root Contribution Index

In [ ]:
"""
Root Contribution Index (RCI):} Calculates the \textbf{contribution of each root}
to the current population by counting the number of descendants for each root,
and scales it by the \textbf{age of the root} (in generations). Calculated for
each generation.
"""

# get the roots based on the condition that they have no incoming edges
roots = [n for n in G.nodes() if G.in_degree(n) == 0]

# calculate the number of leaves that they lead to, for each root
root_contributions = []
for root in roots:
    descs = nx.descendants(G, root)
    # leaf_descs = [n for n in descs if G.nodes[n]["pop"] == max_pop]
    root_age = max_pop - G.nodes[root]["pop"]
    contribution = len(descs) / root_age
    root_contributions.append(contribution)

root_contributions = np.array(root_contributions)
final_rci = root_contributions.sum() / G.number_of_nodes()

final_rci

In [ ]:
phylo_idx = final_mpd / final_rci

phylo_idx